# Question 6

In [ ]:
import numpy as np
from tqdm import tqdm # Nice Progress bar for loops
import matplotlib.pyplot as plt
from Rankine_solver import define_empty_variables, solve_r_rankine_cycle
from Rankine_solver import display_tables, plot_Ts_cycle

In [ ]:
def get_q_from_T(T2, T4):
    vars = define_empty_variables()
    vars['2']['T'] = T2 # C
    vars['4']['T'] = T4 # C

    solved = solve_r_rankine_cycle(vars)
    return solved['qh'], solved['qc']



def get_m_dot_from_T(G,TC,TH,T2, T4, qc, qh):
    if T2 >= TC :
        return 0  # No heat absorption or rejection possible if working fluid(T2) is hotter than cold reservoir (TC)
    m_dot_H = 0
    if abs(qh) > 1e-5 :
        m_dot_H = (G * 1e-3 * np.abs(TH - T4)) / qh

    m_dot_C = 0
    if abs(qc) > 1e-5 :
        m_dot_C = (G * 1e-3 * np.abs(TC - T2)) / qc

    m_dot = max(min(m_dot_C, m_dot_H), 0)
    return m_dot

G=4895.423478997306 # W/K
TC=10  # C
TH=22  # C

# === CONSTANTS ===

''' T2_Range has to be changed to -10C to 35C when Romeo fixes code'''

T2_range = np.linspace(-10, 35, 100)  # 100 points between -10°C and 35°C 

T4_C = 35
def get_W_c(T2_C):
    vars = define_empty_variables()
    vars['2']['T'] = T2_C  # C
    vars['4']['T'] = T4_C # C
    vars['Qh_dot']= 8.18 #kW
    solved = solve_r_rankine_cycle(vars)
    m_dot = solved['m_dot']
    Wc_dot = solved['Wc_dot']
    qc = solved['qc']
    qh = solved['qh']
    
    return(Wc_dot,m_dot,qc,qh)

# === Main Calculation ===
T2_list = []
W_c_list = []
m_dot_list = []
solid_line = []

for T2_C in T2_range:
    Wc_dot,m_dot,qc,qh = get_W_c(T2_C)
    #Store
    m_dot_max = get_m_dot_from_T(G,TC,TH,T2_C,T4_C,qc,qh)
    T2_list.append(T2_C)  # back to Celsius
    W_c_list.append(float(Wc_dot))    # kW
    m_dot_list.append(float(m_dot))
    solid_line.append(float(m_dot) <= m_dot_max)


# === Plotting ===
plt.figure(figsize=(10,6))

T2_list = np.array(T2_list)
W_c_list = np.array(W_c_list)
solid_line = np.array(solid_line)

# Solid line where m_dot <= m_dot_max
plt.plot(T2_list[solid_line], W_c_list[solid_line], 'b-', label='ṁ ≤ ṁ_max')

# Dashed line where m_dot > m_dot_max
plt.plot(T2_list[~solid_line], W_c_list[~solid_line], 'b--', label='ṁ > ṁ_max')

# Mark the transition point
if np.any(~solid_line):
    idx_transition = np.argmax(~solid_line)
    plt.plot(T2_list[idx_transition], W_c_list[idx_transition], 'ro', label='ṁ = ṁ_max')
    print(T2_list[idx_transition], W_c_list[idx_transition])

plt.xlabel('T₂ (°C)')
plt.ylabel('Compressor Input Power Ẇc (kW)')
plt.title('Compressor Input Power vs T₂')
plt.grid(True)
plt.legend()
plt.show()

# === Discussion ===
min_idx = np.argmin(W_c_list)
print(f"Minimal power consumption: {W_c_list[min_idx]:.2f} kW at T₂ = {T2_list[min_idx]:.2f} °C")

if np.isclose(T2_list[min_idx], T4_C, atol=0.5):
    print("T₂ approaches T₄: compression work theoretically approaches zero.")
else:
    print("T₂ is significantly lower than T₄.")


In [ ]:
get_m_dot_from_T(G,TC,TH,9, T4_C),get_W_c(9)